In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import sys
import time
import h5py
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop, SGD, Adam
import tensorflow as tf

from google.colab import drive

In [ ]:
dataset = None
np.set_printoptions(linewidth=250)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for file in sorted(os.listdir('/content/drive/MyDrive/records')):
    if '.csv' in file:
        print(file)
        matrix = np.array(np.genfromtxt('/content/drive/MyDrive/records/' + file, delimiter=',', dtype="int32"))
        matrix = np.delete(matrix, -1, 1)
        desired_output = np.zeros(4)
        if 'relaxed' in file:
            desired_output[0] = 1
        elif 'stimulated' in file:
            desired_output[1] = 1
        elif 'neutral' in file:
            desired_output[2] = 1
        elif 'negative' in file:
            desired_output[3] = 1

        matrix = np.concatenate((matrix, np.tile(desired_output, (np.shape(matrix)[0], 1))), axis=1)
        if dataset is None:
            dataset = np.array(matrix)
        else:
            dataset = np.append(dataset, matrix, axis=0)

print(dataset)
np.savetxt('ioio.csv', dataset, delimiter=',', fmt='%i')

bhargavi_negative_87.csv
bhargavi_neutral_104.csv
bhargavi_relaxed_72.csv
bhargavi_stimulated_138.csv
mohith_negative_100.csv
mohith_neutral_100.csv
mohith_relaxed_100.csv
mohith_stimulated_100.csv
mohith_stimulated_120.csv
mythili_negative_100.csv
mythili_neutral_100.csv
mythili_relaxed_100.csv
mythili_stimulated_100.csv
supraja_negative_67.csv
supraja_neutral_67.csv
supraja_relaxed_68.csv
supraja_stimulated_73.csv
[[6.6201650e+06 4.8606720e+06 3.9859200e+05 ... 0.0000000e+00 0.0000000e+00 1.0000000e+00]
 [8.6819880e+06 1.1276810e+06 4.5811200e+06 ... 0.0000000e+00 0.0000000e+00 1.0000000e+00]
 [1.1113475e+07 6.4450570e+06 7.6884480e+06 ... 0.0000000e+00 0.0000000e+00 1.0000000e+00]
 ...
 [1.1731209e+07 1.1591680e+06 5.9066880e+06 ... 1.0000000e+00 0.0000000e+00 0.0000000e+00]
 [2.6127360e+06 1.1545088e+07 3.1459840e+06 ... 1.0000000e+00 0.0000000e+00 0.0000000e+00]
 [1.6572678e+07 6.5050000e+05 3.6311040e+06 ... 1.0000000e+00 0.0000000e+00 0.0000000e+00]]


In [ ]:
names = ['mohith', 'mythili', "bhargavi", "supraja"]
task = ['relaxed', 'stimulated', 'neutral', 'negative']
spikeBounds = [945*500, 130*500, 43*500, 38*500, 27*500, 22.5*500, 10.97*500, 10.95*500]
dataset = None
np.set_printoptions(linewidth=250)

In [ ]:
dataset=np.genfromtxt('ioio.csv',delimiter=',',dtype=np.int32)

dataset=np.concatenate((dataset[:,0:8],dataset[:,-4:]),axis=1)
def plot_spikes_distribution():
    for wave in range(np.shape(dataset)[1]):
        cazzo=np.zeros(3000)
        i=0
        for bound in range(500, 1500000, 500):
            col = np.copy(dataset[:, wave])
            for mamt in range(np.shape(col)[0]):
                if col[mamt]>=bound:
                    col[mamt]=1
                else:
                    col[mamt]=0

            cazzo[i]=np.sum(col)
            i+=1
        plt.plot(cazzo)
        plt.show()

bounds=[]

In [ ]:
def create_spike_dataset():
    for c in range(8):
        for lamadonnaeputtana in range(np.shape(dataset)[0]):
            if dataset[lamadonnaeputtana][c] > spikeBounds[c]:
                dataset[lamadonnaeputtana][c]=1
            else:
                dataset[lamadonnaeputtana][c]=0

#plot_spikes_distribution()
create_spike_dataset()
np.savetxt('spikeDataset.csv', dataset, fmt='%i', delimiter=',')

In [ ]:
dataset=np.genfromtxt('ioio.csv',delimiter=',',dtype=np.int32)

In [ ]:
def create_array_task(dataset, sequences):
    current_output = dataset[0][-4:]
    array_task = np.zeros(sequences,dtype=np.int32)
    line = 0
    i = 1
    for row in dataset:
        actual = row[-4:]

        if not np.array_equal(actual, current_output):
            array_task[i] = line
            i += 1
            current_output = actual
        line += 1
    return array_task

In [ ]:
def two_over_thirteen_fold_CV(dataset, increment, test_people):
    seq_step = 4
    i = 0

    seqs = create_array_task(dataset, 52)
    train_all = [[-1 for x in range(11)] for y in range(len(dataset))]
    validate_all = [[-1 for x in range(11)] for y in range(len(dataset))]

    while i < (11 * seq_step):
        val_start = seqs[i]
        val_end = seqs[i + (test_people * 4)]
        train_all[int(i / (seq_step * increment))][:val_start] = dataset[:val_start - 1]
        train_all[int(i / (seq_step * increment))][val_start + 1:] = dataset[val_end + 1:]
        validate_all[int(i / (seq_step * increment))] = dataset[val_start:val_end]
        i += (seq_step * increment)
    return train_all, validate_all

In [ ]:
!mkdir cross_validation

mkdir: cannot create directory ‘cross_validation’: File exists


In [ ]:
train, val = two_over_thirteen_fold_CV(dataset, 1, 2)   #test_people default is 2, increment can be 1 or 2.
print(val)
for i in range(11):
    np.savetxt('cross_validation/training_'+str(i)+'.csv', train[i], fmt='%i', delimiter=',')
    np.savetxt('cross_validation/testing_'+str(i)+'.csv', val[i], fmt='%i', delimiter=',')

[array([[ 6620165,  4860672,   398592, ...,        0,        0,        1],
       [ 8681988,  1127681,  4581120, ...,        0,        0,        1],
       [11113475,  6445057,  7688448, ...,        0,        0,        1],
       ...,
       [ 1158912,  8108288, 15873792, ...,        1,        0,        0],
       [ 4617472, 13217792,  1520896, ...,        1,        0,        0],
       [ 5390592,  9547520, 15946752, ...,        1,        0,        0]], dtype=int32), array([[15485440,   920577, 11430656, ...,        0,        0,        1],
       [11877897,  3703041,  5465856, ...,        0,        0,        1],
       [12245504, 14750209,   736768, ...,        0,        0,        1],
       ...,
       [ 8108037, 12510977,  3944192, ...,        1,        0,        0],
       [ 6767363, 11951361,  4395520, ...,        1,        0,        0],
       [14916864,  5106432,  8155136, ...,        1,        0,        0]], dtype=int32), array([], shape=(0, 11), dtype=int32), array([], shape=(0

In [ ]:
np.set_printoptions(linewidth=200)
epochs = 10
binClass = True

batch_size = 5
dataset = np.genfromtxt('spikeDataset.csv', delimiter=',', dtype=np.int32)

In [ ]:
def calculate_max_sequence_length(dataset):
    max_seq_length = 0
    cur_seq_length = 0
    if binClass:
        print(dataset[0])
        current_output = dataset[0][-1]
    else:
        current_output = dataset[0][-4:]
    seqs = 1
    for row in dataset:
        if binClass:
            o = row[-1]
        else:
            o = row[-4:]
        if not np.array_equal(o, current_output):
            seqs += 1
            if cur_seq_length > max_seq_length:
                max_seq_length = cur_seq_length
            current_output = o
            cur_seq_length = 1
        else:
            cur_seq_length += 1
    return max_seq_length, seqs

In [ ]:
def create_array_task1(dataset, sequences):
    current_output = dataset[0][-1]
    array_task = np.zeros(sequences, dtype=np.int32)
    line = 0
    i = 1
    for row in dataset:
        actual = row[-1]

        if not np.array_equal(actual, current_output):
            array_task[i] = line
            i += 1
            current_output = actual
        line += 1
    return array_task

In [ ]:
def create_array_task(dataset, sequences):
    current_output = dataset[0][-4:]
    array_task = np.zeros(sequences, dtype=np.int32)
    line = 0
    i = 1
    for row in dataset:
        actual = row[-4:]

        if not np.array_equal(actual, current_output):
            array_task[i] = line
            i += 1
            current_output = actual
        line += 1
    return array_task

In [ ]:
def k_fold_CV(dataset, folds):
    folds += 1
    fold_len = int(len(dataset) / folds)
    train_all = [[0 for x in range(folds)] for y in range(len(dataset) - fold_len)]
    validate_all = [[0 for x in range(folds)] for y in range(fold_len)]
    for i in range(1, folds):
        val_start = i * fold_len
        val_start = i * fold_len
        val_end = (i + 1) * fold_len
        train_all[i][:val_start] = dataset[:val_start - 1]
        train_all[i][val_start + 1:] = dataset[val_end + 1:]
        validate_all[i] = dataset[val_start:val_end]
    return train_all, validate_all

In [ ]:
def create_model():
    model = Sequential()
    model.add(LSTM(8, stateful=True, return_sequences=True))
    #model.add(LSTM(8, return_sequences=True))
    #model.add(Dropout(.1))
    model.add(LSTM(8))
    #model.add(Dense(8))
    if binClass:
        model.add(Dense(1, activation='sigmoid'))
    else:
        model.add(Dense(4, activation='softmax'))
    return model

In [ ]:
def calculate_accuracy(set, model):
    m, n_test_sequences = calculate_max_sequence_length(set)
    if binClass:
        indices = create_array_task1(set, n_test_sequences)
    else:
        indices = create_array_task(set, n_test_sequences)
    current_sequence_test = 0
    total_accuracy = 0
    for seqa in range(n_test_sequences - 1):
        starta = indices[current_sequence_test]
        enda = indices[current_sequence_test + 1]

        if binClass:
            y_true = set[starta, -1]
            #batch_x = np.expand_dims(set[starta:enda, 0:-1], 0)
        else:
            y_true = np.reshape(set[starta, -4:], (1, 4))
            #batch_x = np.expand_dims(set[starta:enda, 0:-4], 0)
        k = 0
        true_positives = 0
        while k < (enda - starta) / batch_size:
            if binClass:
                batch_x = np.expand_dims(set[starta + k * batch_size:starta + ((k + 1) * batch_size), 0:-1], 0)
            else:
                batch_x = np.expand_dims(set[starta + k * batch_size:starta + ((k + 1) * batch_size), 0:-4], 0)
            output = model.predict_on_batch(batch_x)
            if binClass:
                if (output[0] > 0.5 and y_true == 1) or (output[0] < 0.5 and y_true == 0):
                    true_positives += 1
            else:
                r_output = np.zeros((1, 4))
                r_output[0][np.argmax(output)] = 1
                if np.array_equal(r_output, y_true):
                    true_positives += 1

            k += 1
        # print('accuracy : ',end='')
        total_accuracy += true_positives / ((enda - starta) / batch_size)
        model.reset_states()
        current_sequence_test += 1

    return total_accuracy / n_test_sequences

In [ ]:
max_length, sequences = calculate_max_sequence_length(dataset)
if binClass:
    sequences_indices = create_array_task1(dataset, sequences=sequences)
else:
    sequences_indices = create_array_task(dataset, sequences=sequences)

[1 1 1 1 1 1 1 0 0 0 0 1]


In [ ]:
test_set = dataset[sequences_indices[-4]:]
dataset = dataset[:sequences_indices[-4]]

max_length, sequences = calculate_max_sequence_length(dataset)

network = create_model()
print('Compiling model..')
lr = 0.0005
opt = RMSprop(learning_rate=lr)
network.compile(optimizer=opt, loss='categorical_crossentropy',
                metrics=['accuracy'])
print('Model compiled. Specs:')
network.build(input_shape=(1,5,8))
network.summary()

if not os.path.isdir('./tuning_logs'):
    os.makedirs('./tuning_logs')

log_name = 'u_' + str(epochs) + '_' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H-%M-%S')

old_stdoud = sys.stdout
f = open('tuning_logs/' + log_name + '.txt', 'w+')
sys.stdout = f
print(network.summary())
f.close()
sys.stdout = old_stdoud

[1 1 1 1 1 1 1 0 0 0 0 1]
Compiling model..
Model compiled. Specs:
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (1, 5, 8)                 544       
                                                                 
 lstm_7 (LSTM)               (1, 8)                    544       
                                                                 
 dense_3 (Dense)             (1, 1)                    9         
                                                                 
Total params: 1097 (4.29 KB)
Trainable params: 1097 (4.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
final_accuracy = 0
for i in range(2):
    f = open('tuning_logs/' + log_name + '.txt', 'a')
    f.write('\nFold ' + str(i))
    f.close()
    network = create_model()

    opt = RMSprop(learning_rate=0.0015)
    network.compile(optimizer=opt, loss='mean_absolute_error',
                    metrics=['accuracy'])  # mean_absolute_error or squared
    print('Fold number: ' + str(i))
    X = np.genfromtxt('cross_validation/training_' + str(i) + '.csv', delimiter=',', dtype=np.int32)
    Y = np.genfromtxt('cross_validation/testing_' + str(i) + '.csv', delimiter=',', dtype=np.int32)
    for e in range(epochs):
        current_sequence = 0
        epoch_accuracy = 0
        print('Epoch :' + str(e))
        max_length, sequences = calculate_max_sequence_length(X)
        max_acc = 0
        for seq in range(sequences - 1):
            # print('sequenza'+str(current_sequence))
            start = sequences_indices[current_sequence]
            end = sequences_indices[current_sequence + 1]
            Train = X[start:end]
            if binClass:
                y_true = np.reshape(Train[0][-1], (1, 1))
            else:
                y_true = np.reshape(Train[0][-4:], (1, 4))
            j = 0
            err = 0
            acc = 0
            while j < (end - start) / batch_size:
                if binClass:
                    batch_x = Train[j * batch_size:(j + 1) * batch_size, 0:-1]
                else:
                    batch_x = Train[j * batch_size:(j + 1) * batch_size, 0:-4]
                batch_x = np.expand_dims(batch_x, 0)


                loss, accuracy = network.train_on_batch(tf.cast(batch_x, tf.float32), tf.cast(y_true, tf.float32))
                # print('Accuracy: '+str(accuracy))
                err += loss
                acc += accuracy
                # print('loss on batch '+str(j),end=' ')
                # print(':'+str(loss))
                j += 1
            # print('errore medio per questa sequenza '+str(err/((end-start)/5)))
            epoch_accuracy += acc / ((end - start) / batch_size)
            # print('accuratezza media per questa sequenza '+str(acc/((end-start)/5)))
            # network.reset_states()
            current_sequence += 1
        acctrain = epoch_accuracy / sequences
        print("item for test", Y)
        acctest = calculate_accuracy(Y, network)
        acc_train = '\naccuratezza media per l\'epoca ' + str(e) + ' sul training set: ' + str(acctrain)
        acc_test = '\naccuratezza sul test set :' + str(acctest)
        print(acc_train)
        print(acc_test)
        f = open('tuning_logs/' + log_name + '.txt', 'a')
        f.write(acc_train)
        f.write(acc_test)
        f.write('---------------------')
        f.close()
        if acctest > max_acc:
            max_acc = acctest
        if e == 4:
            final_accuracy += acctest
            max_acc = 0

Fold number: 0
Epoch :0
[ 6620165  4860672   398592  3146496 16321536 15860480  7078656        0        0        0        1]
item for test [[ 6620165  4860672   398592 ...        0        0        1]
 [ 8681988  1127681  4581120 ...        0        0        1]
 [11113475  6445057  7688448 ...        0        0        1]
 ...
 [ 1158912  8108288 15873792 ...        1        0        0]
 [ 4617472 13217792  1520896 ...        1        0        0]
 [ 5390592  9547520 15946752 ...        1        0        0]]
[ 6620165  4860672   398592  3146496 16321536 15860480  7078656        0        0        0        1]

accuratezza media per l'epoca 0 sul training set: 0.6356272884200981

accuratezza sul test set :0.25079617834394907
Epoch :1
[ 6620165  4860672   398592  3146496 16321536 15860480  7078656        0        0        0        1]
item for test [[ 6620165  4860672   398592 ...        0        0        1]
 [ 8681988  1127681  4581120 ...        0        0        1]
 [11113475  6445057  7688

In [ ]:
from keras.models import load_model
model = load_model('network.h5')

In [49]:
Y

array([[15485440,   920577, 11430656, ...,        0,        0,        1],
       [11877897,  3703041,  5465856, ...,        0,        0,        1],
       [12245504, 14750209,   736768, ...,        0,        0,        1],
       ...,
       [ 8108037, 12510977,  3944192, ...,        1,        0,        0],
       [ 6767363, 11951361,  4395520, ...,        1,        0,        0],
       [14916864,  5106432,  8155136, ...,        1,        0,        0]], dtype=int32)

In [61]:
network.predict([[[23623,11182,19781,18250,8140,10460,4684,11255,44,48]]])

1/1 [==============================] - 0s 133ms/step


array([[0.2449429]], dtype=float32)

In [57]:
X[0].shape

(11,)